In [1]:
using JuMP, Gurobi, CSV, DataFrames, XLSX, Random, StatsBase

## Load data

In [2]:
file_path = "../Raw Data/Supply chain logistics problem2.xlsx"

# Reading "OrderList"
orderList_data, orderList_cols = XLSX.readtable(file_path, "OrderList")
orderList_1 = DataFrame(orderList_data, orderList_cols)

# Reading "FreightRates"
freightRates_data, freightRates_cols = XLSX.readtable(file_path, "FreightRates_dedup")
freightRates_1 = DataFrame(freightRates_data, freightRates_cols)

# Reading "WhCosts"
whCosts_data, whCosts_cols = XLSX.readtable(file_path, "WhCosts")
whCosts = DataFrame(whCosts_data, whCosts_cols)

# Reading "WhCapacities"
whCapacities_data, whCapacities_cols = XLSX.readtable(file_path, "WhCapacities")
whCapacities = DataFrame(whCapacities_data, whCapacities_cols)

# Reading "ProductsPerPlant"
productsPerPlant_data, productsPerPlant_cols = XLSX.readtable(file_path, "ProductsPerPlant")
productsPerPlant = DataFrame(productsPerPlant_data, productsPerPlant_cols)

# Reading "VmiCustomers"
vmiCustomers_data, vmiCustomers_cols = XLSX.readtable(file_path, "VmiCustomers")
vmiCustomers = DataFrame(vmiCustomers_data, vmiCustomers_cols)

# Reading "PlantPorts"
plantPorts_data, plantPorts_cols = XLSX.readtable(file_path, "PlantPorts")
plantPorts = DataFrame(plantPorts_data, plantPorts_cols)

## Clean column names
for df in [orderList_1, freightRates_1, whCosts, whCapacities, productsPerPlant, vmiCustomers, plantPorts]
    rename!(df, [replace(lowercase(name), " " => "_") for name in names(df)])
end

In [3]:
orderList = CSV.read("orderList1.csv", DataFrame)
freightRates = CSV.read("freightRates1.csv", DataFrame);

### Reduce number of transportation

In [4]:
unique_ports = unique(plantPorts[:, :port]);

# # Function to randomly select X rows, with at least one row min_wgh_qty==0 and one max_wgh_qty==99999.99
# function select_rows(df::DataFrame, ports::Vector{String})
#     result_df = DataFrame()
#     for port in ports
#         port_df = df[df.orig_port_cd .== port, :]
#         # Get row with min_wgh_qty==0 and max_wgh_qty==99999.99
#         must_have_row = port_df[(port_df.minm_wgh_qty .< 0.001) .& (port_df.max_wgh_qty .> 99999), :]
#         combined_df = must_have_row
#         # Select additional rows randomly
#         additional_rows_needed = max(0, 8 - nrow(combined_df))
#         if additional_rows_needed > 0
#             random_rows = port_df[shuffle(1:nrow(port_df))[1:additional_rows_needed], :]
#             combined_df = vcat(combined_df, random_rows)
#         end
#         result_df = vcat(result_df, combined_df)
#     end
#     return result_df
# end

# freightRates = select_rows(freightRates_1, map(string, unique_ports));

### Reduce number of orders

In [ ]:
# random_rows = sample(1:nrow(orderList_1), 1000, replace = false)
# orderList = orderList_1[random_rows, :];

## Create data matrices

In [5]:
## u: unit volume of each order (OrderList)
u_i = orderList[!, :unit_quantity]

## w: weight of each order (OrderList)
w_i = orderList[!, :weight]

## day_jk: plant to port time (create ourselves)
## day_kt: port to destination time (FreightRates), t = type of transportation

## capacity_j: capacity of each plant in units (WhCapacity)
capacity_j = sort!(whCapacities, :plant_id)[!, :daily_capacity_]

## cost_j: production cost per unit at each plant (whCosts)
cost_j = sort!(whCosts, :wh)[!, :"cost/unit"]


## cost_jk: the per-unit cost of transportation from plant j to origin port k (PlantPorts)
unique_plants = unique(plantPorts[:, :plant_code])

plant_indices = Dict(plant => idx for (idx, plant) in enumerate(unique_plants))
port_indices = Dict(port => idx for (idx, port) in enumerate(unique_ports))

cost_jk = zeros(Int, length(unique_plants), length(unique_ports))

for row in eachrow(plantPorts)
    plant_idx = plant_indices[row[:plant_code]]
    port_idx = port_indices[row[:port]]
    cost_jk[plant_idx, port_idx] = row[:"cost_(extra_column)"]
end


## r_kt: per-unit cost of transporting from port k using transport t
#### SIDE NOTE: missing PORT1 - add it manually to ensure dimensions match (i.e. k always goes from 1-11) ######

max_rows_per_port = maximum([nrow(freightRates[freightRates.orig_port_cd .== port, :]) for port in unique(freightRates.orig_port_cd)])
r_kt = zeros(Float64, length(unique_ports), max_rows_per_port)

for (i, port) in enumerate(unique_ports)
    port_rates = freightRates[freightRates[:, :orig_port_cd] .== port, :rate]
    r_kt[i, 1:length(port_rates)] = port_rates
end

## maxweight_kt: maximum weight capacities from port k, using transport t
maxweight_kt = zeros(Float64, length(unique_ports), max_rows_per_port)
for (i, port) in enumerate(unique_ports)
    port_maxweights = freightRates[freightRates[:, :orig_port_cd] .== port, :max_wgh_qty]
    maxweight_kt[i, 1:length(port_maxweights)] = port_maxweights
end

## minweight_kt: minimum weight capacities from port k, using transport t
minweight_kt = zeros(Float64, length(unique_ports), max_rows_per_port)
for (i, port) in enumerate(unique_ports)
    port_minweights = freightRates[freightRates[:, :orig_port_cd] .== port, :minm_wgh_qty]
    minweight_kt[i, 1:length(port_minweights)] = port_minweights
end

## orderCustomer_li: binary matrix, 1 if customer l did order i (orderList)

unique_customers = unique(orderList.customer)
unique_order_ids = unique(orderList.order_id)
orderCustomer_li = zeros(Int, length(unique_customers), length(unique_order_ids))

customer_indices = Dict(customer => idx for (idx, customer) in enumerate(unique_customers))
order_indices = Dict(order_id => idx for (idx, order_id) in enumerate(unique_order_ids))

for row in eachrow(orderList)
    customer_idx = customer_indices[row.customer]
    order_idx = order_indices[row.order_id]
    orderCustomer_li[customer_idx, order_idx] = 1
end

In [6]:
## day_kt: number of days taken to get from port k using transport t
day_kt = zeros(Int, length(unique_ports), max_rows_per_port)
for (i, port) in enumerate(unique_ports)
    transport_times = freightRates[freightRates[:, :orig_port_cd] .== port, :tpt_day_cnt]
    day_kt[i, 1:length(transport_times)] = transport_times
end

In [ ]:
# ## plantCustomer_lj: binary matrix, 1 if plant j can serve customer l (VmiCustomer)

# # Initialize the matrix with 1s
# plantCustomer_lj = ones(Int, length(unique_customers), length(unique_plants))

# # Iterate over the matrix and set the elements to 0 where the customer-plant pair does not exist
# for i in 1:length(uvmiCustomers_datae_customers)
#     for j in 1:length(unique_plants)
#         customer = customer_indices[i]
#         plant = plant_indices[j]
#         # Check if the pair exists in the DataFrame
#         pair_exists = any(Vmicustomers[:, :customers] .== customer .& Vmicustomers[:, :plant_code] .== plant)
#         if !pair_exists
#             customer_plant_matrix[i, j] = 0
#         end
#     end
# end

# customer_plant_matrix


In [7]:
## orderproduct_im: binary matrix, 1 if order i consists of product m

unique_products = unique(productsPerPlant.product_id)
orderproduct_im = zeros(Int, length(unique_order_ids), length(unique_products))

product_indices = Dict(product_id => idx for (idx, product_id) in enumerate(unique_products))

for row in eachrow(orderList)
    order_idx = order_indices[row[:order_id]]
    product_idx = product_indices[row[:product_id]]
    orderproduct_im[order_idx, product_idx] = 1
end

## plantproduct_jm: binary matrix, 1 if plant j produces product m (productsPerPlant)

plantproduct_jm = zeros(Int, length(unique_plants), length(unique_products))

for row in eachrow(productsPerPlant)
    plant_idx = plant_indices[row[:plant_code]]
    product_idx = product_indices[row[:product_id]]
    plantproduct_jm[plant_idx, product_idx] = 1
end

## plantport_jk: binary matrix, 1 if plant j has connection to port k (PlantPorts)
plantport_jk = zeros(Int, length(unique_plants), length(unique_ports))

for row in eachrow(plantPorts)
    plant_idx = plant_indices[row[:plant_code]]
    port_idx = port_indices[row[:port]]
    plantport_jk[plant_idx, port_idx] = 1
end

## Build model

In [8]:
num_orders = length(unique_order_ids)
num_plants = length(unique_plants)
num_ports = length(unique_ports)
num_transport = max_rows_per_port;

In [9]:
num_orders * num_plants * num_ports * num_transport

1672000

In [ ]:
# CSV.write("freightRates1.csv", freightRates)
# CSV.write("orderList1.csv", orderList)

### Model 1: no considerations for time

In [14]:
model1 =  Model(Gurobi.Optimizer)

@variable(model1, x[1:num_orders, 1:num_plants], Bin)
@variable(model1, y[1:num_orders, 1:num_plants, 1:num_ports], Bin)
@variable(model1, z[1:num_orders, 1:num_plants, 1:num_ports, 1:num_transport], Bin)

## Production 

## All orders are sent to a plant
@constraint(model1, [i = 1:num_orders], sum(x[i, j] for j in 1:num_plants) == 1)

## Plant capacity constraints
@constraint(model1, [j = 1:num_plants], sum(x[i, j] for i in 1:num_orders) <= capacity_j[j])

# ## Orders can only go through existing plant-product mappings
# order_plant = orderproduct_im * plantproduct_jm'
# @constraint(model1, [i = 1:num_orders, j = 1:num_plants], x[i, j] <= order_plant[i, j])


## Transportation: plant -> port

## What enters a plant exits the plant
@constraint(model1, [j = 1:num_plants], sum(x[i, j] for i in 1:num_orders) == sum(y[i, j, k] for i in 1:num_orders, k in 1:num_ports))

## Orders can only go through existing plant-port mappings
@constraint(model1, plant_port_constraints[i=1:num_orders, j=1:num_plants, k=1:num_ports], y[i, j, k] <= plantport_jk[j, k])


## Transportation: port - > destination

## What enters a port exits the port
@constraint(model1, [k = 1:num_ports], sum(y[i, j, k] for i in 1:num_orders, j in 1:num_plants) == sum(z[i, j, k, t] for i in 1:num_orders, j in 1:num_plants, t in 1:num_transport))

## Maximum weight constraints
@constraint(model1, [k = 1:num_ports, t = 1:num_transport], sum(z[i, j, k, t] * w_i[i] for i in 1:num_orders, j in 1:num_plants) <= maxweight_kt[k, t])

## Minimum weight constraints
@constraint(model1, [k = 1:num_ports, t = 1:num_transport], sum(z[i, j, k, t] * w_i[i] for i in 1:num_orders, j in 1:num_plants) >= minweight_kt[k, t])

lambda = 0.5

# # Objective: with time
# @objective(model1, Min, sum(x[i, j] * u_i[i] * cost_j[j] for i in 1:num_orders, j in 1:num_plants) + 
# sum(y[i, j, k] * u_i[i] * cost_jk[j, k] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports) + 
# sum(z[i, j, k, t] * w_i[i] * r_kt[k, t] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports, t in 1:num_transport) + 
# lambda * sum(z[i, j, k, t] * day_kt[k, t] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports, t in 1:num_transport))

# Objective: without time
@objective(model1, Min, sum(x[i, j] * u_i[i] * cost_j[j] for i in 1:num_orders, j in 1:num_plants) + 
sum(y[i, j, k] * u_i[i] * cost_jk[j, k] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports) + 
sum(z[i, j, k, t] * w_i[i] * r_kt[k, t] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports, t in 1:num_transport))

# @objective(model1, Min, 0)

optimize!(model1)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-10
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 210225 rows, 1900000 columns and 5700000 nonzeros
Model fingerprint: 0xbdbf2d42
Variable types: 0 continuous, 1900000 integer (1900000 binary)
Coefficient statistics:
  Matrix range     [1e-02, 9e+02]
  Objective range  [4e-04, 3e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 1e+05]
Presolve removed 209004 rows and 1800679 columns
Presolve time: 1.72s
Presolved: 1221 rows, 99321 columns, 235598 nonzeros
Variable types: 12 continuous, 99309 integer (30892 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.03s

Solved with dual simplex

Root relaxation: objective 1.648574e+06, 3634 iterations, 0.24 seconds (0.38 work units)


In [23]:
x_no_time = JuMP.value.(x)
y_no_time = JuMP.value.(y)
z_no_time = JuMP.value.(z)
objective_no_time = JuMP.objective_value.(model1);

In [24]:
## Monetary cost, without optimizing for time
objective_no_time

1.6515731729528483e6

In [25]:
## Time taken, without optimizing for time
time_no_opt = sum(z_no_time[i, j, k, t] * day_kt[k, t] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports, t in 1:num_transport)

4818.0

### Model 2: optimizing time and costs simultaneously

In [26]:
model2 =  Model(Gurobi.Optimizer)

@variable(model2, x[1:num_orders, 1:num_plants], Bin)
@variable(model2, y[1:num_orders, 1:num_plants, 1:num_ports], Bin)
@variable(model2, z[1:num_orders, 1:num_plants, 1:num_ports, 1:num_transport], Bin)

## Production 

## All orders are sent to a plant
@constraint(model2, [i = 1:num_orders], sum(x[i, j] for j in 1:num_plants) == 1)

## Plant capacity constraints
@constraint(model2, [j = 1:num_plants], sum(x[i, j] for i in 1:num_orders) <= capacity_j[j])

# ## Orders can only go through existing plant-product mappings
# order_plant = orderproduct_im * plantproduct_jm'
# @constraint(model1, [i = 1:num_orders, j = 1:num_plants], x[i, j] <= order_plant[i, j])


## Transportation: plant -> port

## What enters a plant exits the plant
@constraint(model2, [j = 1:num_plants], sum(x[i, j] for i in 1:num_orders) == sum(y[i, j, k] for i in 1:num_orders, k in 1:num_ports))

## Orders can only go through existing plant-port mappings
@constraint(model2, plant_port_constraints[i=1:num_orders, j=1:num_plants, k=1:num_ports], y[i, j, k] <= plantport_jk[j, k])


## Transportation: port - > destination

## What enters a port exits the port
@constraint(model2, [k = 1:num_ports], sum(y[i, j, k] for i in 1:num_orders, j in 1:num_plants) == sum(z[i, j, k, t] for i in 1:num_orders, j in 1:num_plants, t in 1:num_transport))

## Maximum weight constraints
@constraint(model2, [k = 1:num_ports, t = 1:num_transport], sum(z[i, j, k, t] * w_i[i] for i in 1:num_orders, j in 1:num_plants) <= maxweight_kt[k, t])

## Minimum weight constraints
@constraint(model2, [k = 1:num_ports, t = 1:num_transport], sum(z[i, j, k, t] * w_i[i] for i in 1:num_orders, j in 1:num_plants) >= minweight_kt[k, t])

lambda = 0.5

# Objective: with time
@objective(model2, Min, sum(x[i, j] * u_i[i] * cost_j[j] for i in 1:num_orders, j in 1:num_plants) + 
sum(y[i, j, k] * u_i[i] * cost_jk[j, k] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports) + 
sum(z[i, j, k, t] * w_i[i] * r_kt[k, t] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports, t in 1:num_transport) + 
lambda * sum(z[i, j, k, t] * day_kt[k, t] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports, t in 1:num_transport))

# # Objective: without time
# @objective(model1, Min, sum(x[i, j] * u_i[i] * cost_j[j] for i in 1:num_orders, j in 1:num_plants) + 
# sum(y[i, j, k] * u_i[i] * cost_jk[j, k] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports) + 
# sum(z[i, j, k, t] * w_i[i] * r_kt[k, t] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports, t in 1:num_transport))

# @objective(model1, Min, 0)

optimize!(model2)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-10
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 210225 rows, 1900000 columns and 5700000 nonzeros
Model fingerprint: 0xf7b1e81c
Variable types: 0 continuous, 1900000 integer (1900000 binary)
Coefficient statistics:
  Matrix range     [1e-02, 9e+02]
  Objective range  [8e-04, 3e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 1e+05]
Presolve removed 209004 rows and 1800679 columns
Presolve time: 2.02s
Presolved: 1221 rows, 99321 columns, 235598 nonzeros
Variable types: 12 continuous, 99309 integer (30892 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.03s

Solved with dual simplex

Root relaxation: objective 1.649203e+06, 3739 iterations, 0.26 seconds (0.40 work units)


In [27]:
x_with_time = JuMP.value.(x)
y_with_time = JuMP.value.(y)
z_with_time = JuMP.value.(z)
objective_with_time = JuMP.objective_value.(model2);

In [28]:
## Monetary cost, when optimizing simultaneously with time
monetary_with_time = sum(x_with_time[i, j] * u_i[i] * cost_j[j] for i in 1:num_orders, j in 1:num_plants) + 
sum(y_with_time[i, j, k] * u_i[i] * cost_jk[j, k] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports) + 
sum(z_with_time[i, j, k, t] * w_i[i] * r_kt[k, t] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports, t in 1:num_transport)

1.6515921729569982e6

In [29]:
## Time taken, when optimizing for time
time_with_opt = sum(z_with_time[i, j, k, t] * day_kt[k, t] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports, t in 1:num_transport)

1253.0

In [28]:
# Initialize an empty DataFrame to store the results
results_df = DataFrame(lambda = Float64[], monetary_with_time = Float64[], time_with_opt = Float64[])

# Lambda values to iterate over
lambda_values = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

for lambda in lambda_values

    model2 =  Model(Gurobi.Optimizer)

    @variable(model2, x[1:num_orders, 1:num_plants], Bin)
    @variable(model2, y[1:num_orders, 1:num_plants, 1:num_ports], Bin)
    @variable(model2, z[1:num_orders, 1:num_plants, 1:num_ports, 1:num_transport], Bin)

    ## Production 

    ## All orders are sent to a plant
    @constraint(model2, [i = 1:num_orders], sum(x[i, j] for j in 1:num_plants) == 1)

    ## Plant capacity constraints
    @constraint(model2, [j = 1:num_plants], sum(x[i, j] for i in 1:num_orders) <= capacity_j[j])

    # ## Orders can only go through existing plant-product mappings
    # order_plant = orderproduct_im * plantproduct_jm'
    # @constraint(model1, [i = 1:num_orders, j = 1:num_plants], x[i, j] <= order_plant[i, j])


    ## Transportation: plant -> port

    ## What enters a plant exits the plant
    @constraint(model2, [j = 1:num_plants], sum(x[i, j] for i in 1:num_orders) == sum(y[i, j, k] for i in 1:num_orders, k in 1:num_ports))

    ## Orders can only go through existing plant-port mappings
    @constraint(model2, plant_port_constraints[i=1:num_orders, j=1:num_plants, k=1:num_ports], y[i, j, k] <= plantport_jk[j, k])


    ## Transportation: port - > destination

    ## What enters a port exits the port
    @constraint(model2, [k = 1:num_ports], sum(y[i, j, k] for i in 1:num_orders, j in 1:num_plants) == sum(z[i, j, k, t] for i in 1:num_orders, j in 1:num_plants, t in 1:num_transport))

    ## Maximum weight constraints
    @constraint(model2, [k = 1:num_ports, t = 1:num_transport], sum(z[i, j, k, t] * w_i[i] for i in 1:num_orders, j in 1:num_plants) <= maxweight_kt[k, t])

    ## Minimum weight constraints
    @constraint(model2, [k = 1:num_ports, t = 1:num_transport], sum(z[i, j, k, t] * w_i[i] for i in 1:num_orders, j in 1:num_plants) >= minweight_kt[k, t])

    # Objective: with time
    @objective(model2, Min, (1 - lambda) * (sum(x[i, j] * u_i[i] * cost_j[j] for i in 1:num_orders, j in 1:num_plants) + 
    sum(y[i, j, k] * u_i[i] * cost_jk[j, k] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports) + 
    sum(z[i, j, k, t] * w_i[i] * r_kt[k, t] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports, t in 1:num_transport)) + 
    lambda * sum(z[i, j, k, t] * day_kt[k, t] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports, t in 1:num_transport))

    optimize!(model2)

    # Extract the solution and compute the required values
    x_with_time = JuMP.value.(x)
    y_with_time = JuMP.value.(y)
    z_with_time = JuMP.value.(z)
    
    monetary_cost = sum(x_with_time[i, j] * u_i[i] * cost_j[j] for i in 1:num_orders, j in 1:num_plants) + 
    sum(y_with_time[i, j, k] * u_i[i] * cost_jk[j, k] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports) + 
    sum(z_with_time[i, j, k, t] * w_i[i] * r_kt[k, t] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports, t in 1:num_transport)

    time_taken = sum(z_with_time[i, j, k, t] * day_kt[k, t] for i in 1:num_orders, j in 1:num_plants, k in 1:num_ports, t in 1:num_transport)

    push!(results_df, (lambda, monetary_cost, time_taken))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-10
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 210225 rows, 1900000 columns and 5700000 nonzeros
Model fingerprint: 0xbdbf2d42
Variable types: 0 continuous, 1900000 integer (1900000 binary)
Coefficient statistics:
  Matrix range     [1e-02, 9e+02]
  Objective range  [4e-04, 3e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 1e+05]
Presolve removed 209004 rows and 1800679 columns
Presolve time: 2.07s
Presolved: 1221 rows, 99321 columns, 235598 nonzeros
Variable types: 12 continuous, 99309 integer (30892 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.05s

Solved with dual simplex

Root relaxation: objective 1.648574e+06, 3634 iterations, 0.26 seconds (0.38 work units)


In [29]:
results_df

Row,lambda,monetary_with_time,time_with_opt
,Float64,Float64,Float64
1,0.0,1.65157e6,4818.0
2,0.1,1.65158e6,1356.0
3,0.2,1.65159e6,1260.0
4,0.3,1.65158e6,1253.0
5,0.4,1.65158e6,1253.0
6,0.5,1.6516e6,1253.0
7,0.6,1.65164e6,1256.0
8,0.7,1.6516e6,1253.0
9,0.8,1.6516e6,1252.0


In [30]:
CSV.write("Costs with lambda.csv", results_df)

"Costs with lambda.csv"

In [ ]:
using Plots

In [26]:
# #Plot for Monetary Cost
# plot1 = plot(results_df[!, :lambda], results_df[!, :monetary_with_time], 
#      title="Monetary Cost vs Lambda", 
#      xlabel="Lambda", ylabel="Monetary Cost ($)", 
#      legend=false)

# display(plot1)



In [27]:
# # Plot for Time Taken
# plot2 = plot(results_df[!, :lambda], results_df[!, :time_with_opt], 
#      title="Time Taken vs Lambda", 
#      xlabel="Lambda", ylabel="Time Taken (days)", 
#      legend=false)

# # Display the plot
# display(plot2)